<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Session_10_Traders_move_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1- Review lesson 9 and intro

Script Notes: 

In lesson 9 we introduced the dynamics of agent sequencing and began the move function with agents finding all their neighbors without an agent. In this lesson we are going caluclate of those options which one is best and closest. 

Remember if you shut down your colab kernel you will need to add the sugar-map.txt file again or you can mount your drive and not have to load it every time   

*Review each class and model set up*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
  import mesa
import numpy as np
import matplotlib.pyplot as plt



%matplotlib inline 

     |████████████████████████████████| 2.5 MB 7.6 MB/s 
     |████████████████████████████████| 596 kB 60.8 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 


In [3]:
class Sugar(mesa.Agent):
    """
    Sugar is a FSM that
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn (rule G1).
    """
    
    
    def __init__(self, unique_id, model, pos, max_sugar): 
      super().__init__(unique_id, model) #part 3
      self.pos = pos
      self.amount = max_sugar
      self.max_sugar = max_sugar

    def step(self): #Part 1
      self.amount = min([self.max_sugar,self.amount+1])
      
      

In [4]:
class Spice(mesa.Agent):
    """
    Spice is a FSM that
    - contains an amount of spice
    - grows 1 amount of spice at each turn. (rule G1)
    """

    def __init__(self, unique_id, model, pos, max_spice): 
        super().__init__(unique_id, model) 
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self): #Part 1
      self.amount = min([self.max_spice,self.amount+1])
       

## Add Trader welfare function 

**Part 1**

------------

We now have  list of every cell in our vision that does not already have an agent on it. 

so we want to determine which cell to move to based on which cell will produce the most welfare.  

so we will take our list of neighbors and iterate through each one and then add a calcuate welfare_function but to do this we need to know how much sugar and spice are in the space. At this point are code is getting more complicated so it is important to add comments so people know how the code fits together and also so you can remind yourself. 


So the equation we are using is found on page 97 if you have the book and it is the functional form of the cobb-douglas function. You can see the generalizable function from the [link on the course page](https://inomics.com/terms/cobb-douglas-production-function-1456726#:~:text=A%20Cobb%2DDouglas%20production%20function,technological%20change%20in%20production%20methods.) But what this does is adjust the agents decision making based on how much sugar and spice they have relative to their metabolism  so they make an optimal decision based on their vision.  

we print to make sure it is producing results, but we are only doing the sugar and spice in our position we need to get the sugar and spice in the cell

**Part 2**

--------





In [13]:
class Trader(mesa.Agent):
    """
    TraderAgent is a 
    - has a metabolism for sugar and spice
    - harvest and trades sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model,pos,moore=False, sugar=0, 
                 spice=0, metabolism_sugar=0, metabolism_spice=0,
                 vision=0): 
      super().__init__(unique_id, model)
      self.pos = pos 
      self.moore = False 
      self.sugar = sugar 
      self.spice = spice 
      self.metabolism_sugar=metabolism_sugar 
      self.metabolism_spice=metabolism_spice 
      self.vision = vision 

  
    def is_occupied(self, pos): 
      '''
      Helper function for move
      '''
      
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for a in this_cell:
          if isinstance(a, Trader) and a.pos != self.pos:
              return True
      return False
       
    def calculate_welfare(self, sugar, spice): #This will be built to be like the exmaple later with none, but right now it wouldn't make any sense
      '''
      Helper function for move
      '''

      #calculate total resources
      m_total = self.metabolism_sugar + self.metabolism_spice
      #Cobb-Douglas functional form
      return sugar** (self.metabolism_sugar/m_total) * spice ** (
          self.metabolism_spice/m_total)
    
    
    def move(self): 
           
      # 1. Get neighbors within vision

      neighbors = [i 
                   for i in self.model.grid.get_neighborhood(
                       self.pos, self.moore,True, self.vision
                       )
                   if not self.is_occupied(i)
      ]

      # 2. Find the patch which porduce the maximum welfare. 

      welfares = [ self.calculate_welfare(self.sugar, self.spice) for pos in neighbors] 

      print(welfares)
      
      



## Add Trader randomize and move






In [14]:
class SugarscapeG1mt(mesa.Model):

  def __init__(self, width=50, height=50, initial_population=100,
               endowment_min =25, endowment_max =50, metabolism_min = 1,
                 metabolism_max = 5, vision_min = 1, vision_max =5): 
    
    self.width = width
    self.height = height
    self.initial_population =initial_population 
    self.endowment_min = endowment_min 
    self.endowment_max = endowment_max 
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max


    self.schedule = mesa.time.RandomActivationByType(self)
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

    sugar_distribution = np.genfromtxt("/content/drive/MyDrive/sugar-map.txt") 
    spice_distribution = spice = np.flip(sugar_distribution,1)
    
    #ensure unique id
    agent_id = 0
    for _, x, y in self.grid.coord_iter(): 
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y),max_sugar)
        self.grid.place_agent(sugar, (x, y))
        #Place Agent
        self.schedule.add(sugar) 
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.grid.place_agent(spice,(x,y))
        self.schedule.add(spice)
        agent_id += 1
         
    
    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width) 
      y = self.random.randrange(self.height) 
      # See GAS page 108 for parameters initialization.
      # Each agent is endowed by a random amount of sugar and spice
      sugar = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      spice = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      #add metabolism also on page #108
      metabolism_sugar = self.random.uniform(self.metabolism_min, self.metabolism_max)
      metabolism_spice = self.random.uniform(self.metabolism_min, self.metabolism_max)
      #add vision
      vision =int(self.random.uniform(self.vision_min, self.vision_max))
      trader = Trader(
                agent_id,
                self,
                (x, y),
                False,
                sugar,
                spice,
                metabolism_sugar,
                metabolism_spice,
                vision,
            )
      self.grid.place_agent(trader, (x, y))
      self.schedule.add(trader)
      agent_id += 1

  def step(self): 
    
    for sugar in self.schedule.agents_by_type[Sugar].values(): 
      sugar.step()
    
    for spice in self.schedule.agents_by_type[Spice].values(): 
      spice.step()
    
    Traders = self.schedule.agents_by_type[Trader].values()
    Trader_shuffle = list(Traders)
    self.random.shuffle(Trader_shuffle)
    for agent in Trader_shuffle: 
      agent.move()  
    
    self.schedule.time +=1
    self.schedule.steps += 1
    

  
  def run_model(self, step_count=1000):
    
    for i in range(step_count):
      self.step()
      

    #self.schedule.step()_

        

## Model Run 


In [15]:
model = SugarscapeG1mt()
model.run_model(step_count=1) #part 1



[46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342, 46.14196163465342]
[37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.37772143161903, 37.3777214